In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import pandas as pd
import requests
from bs4 import BeautifulSoup


In [ ]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),
                          chrome_options=options)
infos_main_screen = []

for i in range(1, 12):
    url = f"https://polkastarter.com/projects?page={i}#search"
    driver.get(url)

    f = driver.find_element(
        By.XPATH, '//table[@id="ps--funded-projects__table"]//tbody')
    for name in f.find_elements_by_xpath(".//tr"):

        info = name.find_elements_by_xpath(".//td")
        temp_list = [i.text for i in info]
        print(temp_list)
        infos_main_screen.append(temp_list)
        # breakpoint()


In [ ]:
cols = [
    "Project Name", "Type", "Participants", "Total Raised", "Current Price",
    "ATH Since IDO", "Ended in (UTC)", "Networks"
]

df = pd.DataFrame(infos_main_screen, columns=[cols]).drop_duplicates()

In [ ]:
df[['Project Name', 'Ticker']] = df['Project Name'].str.split("\n",
                                                              expand=True)

In [ ]:
df['Link'] = "https://polkastarter.com/projects/" + (
    df['Project Name'].str.lower()).str.replace(" ", "-").str.replace(
        ".", "-")


In [ ]:
df['Link'] = df['Link'].str.replace("polychain-monsters", "polkamon")
df['Link'] = df['Link'].str.replace("shyft-network-inc-", "shyft-network-inc")


In [ ]:
cont_final = []
for url in df['Link']:
    print(url)
    content = requests.get(url)
    soup = BeautifulSoup(content.content, 'html.parser')
    data = soup.find("div", {"id": "app-content"}).find_next()
    start = str(data).find("{")
    end = str(data).find("}")
    cont_final.append(str(data)[start:end])


In [ ]:

cont_final = pd.Series(cont_final)

a = cont_final.str.replace("\r","").str.replace("\n","")
